# Project Description

## Context & Objective

The second-hand phone business/industry is poised for significant growth in the near to medium future with the IDC (Internation Data Corporation) predicting a whooping \$52.7bn by 2023 with a compound annual growth rate (CAGR) of 13.6% from 2018 to 2023.

There are many advantages to using a used cell phone:  
1. Significant cost savings with warranties
2. Increases the life of the cell phone - Environmentally friendly 

ReCell is a start-up looking to take advantage of the potential boom in the industry and are looking to use ML to understand the dynamics.

Build a linear regression model that predicts the price of a used phone and identify factors that significantly influence the price.  


### Data Information:  
brand_name: Name of manufacturing brand  
os: OS on which the phone runs  
screen_size: Size of the screen in cm  
4g: Whether 4G is available or not  
5g: Whether 5G is available or not  
main_camera_mp: Resolution of the rear camera in megapixels  
selfie_camera_mp: Resolution of the front camera in megapixels  
int_memory: Amount of internal memory (ROM) in GB  
ram: Amount of RAM in GB  
battery: Energy capacity of the phone battery in mAh  
weight: Weight of the phone in grams  
release_year: Year when the phone model was released  
days_used: Number of days the used/refurbished phone has been used  
new_price: Price of a new phone of the same model in euros  
used_price: Price of the used/refurbished phone in euros  

### 1. Import Libraries

In [1]:
import pandas as pd 
import numpy as np 

#Visualization Libraries:
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 

#statistical and regression Libraries 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split # Sklearn package's randomized data splitting function
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.stats.diagnostic as sms

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 200)


import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read the data into the dataframe
df=pd.read_csv('used_phone_data.csv')
df.shape

(3571, 15)

There are 3571 data rows in the data set with 15 columns.

In [3]:
#Taking a look at the first couple of columns 
df.head()

,brand_name,os,screen_size,4g,5g,main_camera_mp,selfie_camera_mp,int_memory,ram,battery,weight,release_year,days_used,new_price,used_price
0,Honor,Android,23.97,yes,no,13.0,5.0,64.0,3.0,3020.0,146.0,2020,127,111.62,86.96
1,Honor,Android,28.10,yes,yes,13.0,16.0,128.0,8.0,4300.0,213.0,2020,325,249.39,161.49
2,Honor,Android,24.29,yes,yes,13.0,8.0,128.0,8.0,4200.0,213.0,2020,162,359.47,268.55
3,Honor,Android,26.04,yes,yes,13.0,8.0,64.0,6.0,7250.0,480.0,2020,345,278.93,180.23
4,Honor,Android,15.72,yes,no,13.0,8.0,64.0,3.0,5000.0,185.0,2020,293,140.87,103.80


In [5]:
# Sampling random data points to see if the observation holds true
np.random.seed(1)
df.sample(n=10) 

,brand_name,os,screen_size,4g,5g,main_camera_mp,selfie_camera_mp,int_memory,ram,battery,weight,release_year,days_used,new_price,used_price
2501,Samsung,Android,13.49,yes,no,13.0,13.0,32.0,4.00,3600.0,181.0,2017,683,198.680,79.47
2782,Sony,Android,13.81,yes,no,NaN,8.0,32.0,4.00,3300.0,156.0,2019,195,198.150,149.10
605,Others,Android,12.70,yes,no,8.0,5.0,16.0,4.00,2400.0,137.0,2015,1048,161.470,48.39
2923,Vivo,Android,19.37,yes,no,13.0,16.0,64.0,4.00,3260.0,149.3,2019,375,211.880,138.31
941,Others,Others,5.72,no,no,0.3,0.3,32.0,0.25,820.0,90.0,2013,883,29.810,8.92
1833,LG,Android,13.49,no,no,8.0,1.3,32.0,4.00,3140.0,161.0,2013,670,240.540,96.18
671,Apple,iOS,14.92,yes,no,12.0,7.0,64.0,4.00,5493.0,48.0,2018,403,700.150,350.08
1796,LG,Android,17.78,yes,no,5.0,0.3,16.0,4.00,4000.0,294.8,2014,708,189.300,75.94
757,Asus,Android,13.49,yes,no,13.0,8.0,32.0,4.00,5000.0,181.0,2017,612,270.500,108.13
3528,Realme,Android,15.72,yes,no,NaN,16.0,64.0,4.00,4035.0,184.0,2019,433,159.885,80.00


The columns show the properties of the phones. The Brand name, whether it is 4G or 5G enabled and the operating system are object data type as is expected. There remaining are numberical data types. It looks like we have some NAN values in the main camera mega pixel information. Looking forward to the linear regression, there are some 'object' variables that I believe would be important variables in determining the price of use phones that should be converted to categorical variavles: OS and connectivity capability   4G and 5G.


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3571 entries, 0 to 3570
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   brand_name        3571 non-null   object 
 1   os                3571 non-null   object 
 2   screen_size       3571 non-null   float64
 3   4g                3571 non-null   object 
 4   5g                3571 non-null   object 
 5   main_camera_mp    3391 non-null   float64
 6   selfie_camera_mp  3569 non-null   float64
 7   int_memory        3561 non-null   float64
 8   ram               3561 non-null   float64
 9   battery           3565 non-null   float64
 10  weight            3564 non-null   float64
 11  release_year      3571 non-null   int64  
 12  days_used         3571 non-null   int64  
 13  new_price         3571 non-null   float64
 14  used_price        3571 non-null   float64
dtypes: float64(9), int64(2), object(4)
memory usage: 418.6+ KB


We can also see that the memory data is also missing some values. So is the battery and weight data missing some information. Lets take a look at the missing values

In [7]:
df.isnull().sum().sort_values(ascending=False)

main_camera_mp      180
int_memory           10
ram                  10
weight                7
battery               6
selfie_camera_mp      2
brand_name            0
os                    0
screen_size           0
4g                    0
5g                    0
release_year          0
days_used             0
new_price             0
used_price            0
dtype: int64

We can see that the main_camera_mp variable to have the most missing data. It would be interesting to see what release years and brand have this issue. There are a couple of options to deal with missing data: (1) drop all the rows with missing data, (2) drop the rear camera MP column (3) Fill in data. 

In [8]:
#Correlation may shed some early light on the imporatance and guide on path to dealing with the missing values 
df.corr()

,screen_size,main_camera_mp,selfie_camera_mp,int_memory,ram,battery,weight,release_year,days_used,new_price,used_price
screen_size,1.000000,0.189274,0.370251,0.143750,0.283320,0.741848,0.634869,0.454596,-0.397210,0.342092,0.385928
main_camera_mp,0.189274,1.000000,0.423704,0.004032,0.206895,0.239659,-0.073371,0.354243,-0.142915,0.360324,0.281065
selfie_camera_mp,0.370251,0.423704,1.000000,0.283188,0.445200,0.344864,-0.005384,0.696703,-0.556027,0.399368,0.497530
int_memory,0.143750,0.004032,0.283188,1.000000,0.079572,0.113488,0.017879,0.219256,-0.228636,0.343227,0.404913
ram,0.283320,0.206895,0.445200,0.079572,1.000000,0.234465,0.069766,0.311379,-0.286884,0.443403,0.522659
battery,0.741848,0.239659,0.344864,0.113488,0.234465,1.000000,0.704468,0.457467,-0.351055,0.341864,0.363767
weight,0.634869,-0.073371,-0.005384,0.017879,0.069766,0.704468,1.000000,0.062348,-0.059272,0.197986,0.170654
release_year,0.454596,0.354243,0.696703,0.219256,0.311379,0.457467,0.062348,1.000000,-0.748594,0.296972,0.456764
days_used,-0.397210,-0.142915,-0.556027,-0.228636,-0.286884,-0.351055,-0.059272,-0.748594,1.000000,-0.235613,-0.470760
new_price,0.342092,0.360324,0.399368,0.343227,0.443403,0.341864,0.197986,0.296972,-0.235613,1.000000,0.926087


Firstly I look at how much of a correlation there is between rear camera pixels and my target variable "used Price", ~0.28. Individually is not a high correlation per se but when compared to the other features we can see that it is not insignificant. so droping the column MAY loose some valuable information. Furthermore, I see that selfie camera has a strong correlation with my target variable so I am tempted to keep anything with cameras. 